In [3]:
#Example 16.1
#Equation (16.2)
using JuMP
using Gurobi
m = Model(solver = GurobiSolver(OutputFlag=0));
@variable(m, Qs >= 0);
@variable(m, Qw >= 0);
@variable(m, R[1:3] >= 0);
@constraint(m, R[1] - Qs == -30);
@constraint(m, R[2] - R[1] == -30);
@constraint(m, R[3] - R[2] == 123);
@constraint(m, Qw - R[3] == 102);
@objective(m, Min, Qs+Qw);
print(m);

Min Qs + Qw
Subject to
 R[1] - Qs = -30
 R[2] - R[1] = -30
 R[3] - R[2] = 123
 Qw - R[3] = 102
 R[i] ≥ 0 ∀ i ∈ {1,2,3}
 Qs ≥ 0
 Qw ≥ 0


In [4]:
#Solve and print results
solve(m);
println("Qs = ", getvalue(Qs));
println("Qw = ", getvalue(Qw));

Academic license - for non-commercial use only
Qs = 60.0
Qw = 225.0


In [5]:
# Structured model, equation (16.5)
# Import data, define sets and dictionary
Data = readcsv("Eg_16.1.csv")
S = ["HP"];
W = ["CW"];
K = length(Data[1,:])-1;
Sk = [["HP"],[],[],[]];
Wk = [[],[],[],["CW"]];
Hk = [[],["H1"],["H1","H2"],["H1","H2"]];
Ck = [["C1"],["C1"],["C1","C2"],["C2"]];
H = ["H1","H2"];
C = ["C1","C2"];
Q = Dict(("H1",1) => 0.1)
for i in 1:4
    for j in 1:4
        Q[(Data[i,1],j)] = Data[i,j+1];
    end
end

In [7]:
m2  = Model(solver = GurobiSolver(OutputFlag=0));
@variable(m2, Qs[S] >= 0);
@variable(m2, Qw[W] >= 0);
@variable(m2, R[0:K] >= 0);
@constraint(m2, R[0] == 0);
@constraint(m2, R[K] == 0);
@constraint(m2, [k in 1:K], R[k] - R[k-1]  - sum(Qs[m] for m in Sk[k]) + sum(Qw[n] for n in Wk[k])
    == sum(Q[i,k] for i in H) - sum(Q[j,k] for j in C));
@objective(m2, Min, sum(Qs[m] for m in S) + sum(Qw[n] for n in W));
print(m2);

Min Qs[HP] + Qw[CW]
Subject to
 R[0] = 0
 R[4] = 0
 R[1] - R[0] - Qs[HP] = -30
 R[2] - R[1] = -30
 R[3] - R[2] = 123
 R[4] - R[3] + Qw[CW] = 102
 Qs[i] ≥ 0 ∀ i ∈ {HP}
 Qw[i] ≥ 0 ∀ i ∈ {CW}
 R[i] ≥ 0 ∀ i ∈ {0,1,2,3,4}


In [8]:
solve(m2);
println("Qs = ", getvalue(Qs));
println("Qw = ", getvalue(Qw));

Academic license - for non-commercial use only
Qs = Qs: 1 dimensions:
[HP] = 60.0
Qw = Qw: 1 dimensions:
[CW] = 225.0
